In [41]:
import pandas as pd
import numpy as np

# Load workshop & mastelab datasets
df_ws_ml = pd.read_excel('Workshops and Masterlabs.xlsx')
df_ws_ml.tail()

,TumoID,Workshop,StartDate,StudentSchedule,Star,PassStatus,Attendance,Comment
12874,190922001271,Stop-motion animated Music video (From Home + ...,28 juin 2021,Friday 17:30,False,Pass,✅✅✅✅✅✅,Montre un grand potentiel et travaille pour l’...
12875,190213001423,Stop-motion animated Music video (From Home + ...,28 juin 2021,Tuesday 17:30,False,Withdrawn,✅✅🔲🔲🔲🔲,Ne souhaite pas participer parce que. Il trouv...
12876,201010000139,Stop-motion animated Music video (From Home + ...,28 juin 2021,Thursday 17:30,False,Withdrawn,🔴🔲🔲🔲🔲🔲,"Avant le début du lab, l'étudiant a décidé de ..."
12877,180718000473,Stop-motion animated Music video (From Home + ...,28 juin 2021,Friday 17:30,False,Pass,✅✅✅✅✅✅,Montre un grand potentiel et travaille pour l’...
12878,181130000196,Stop-motion animated Music video (From Home + ...,28 juin 2021,Wednesday 14:30,False,Pass,✅✅✅✅✅🔴,Montre un grand potentiel et travaille pour l’...


In [42]:
# Filter out masterlabs from workshops
words = ['masterlab', 'rencontre','projet','programme','stop-motion', 'ateliers', 'compose', 'photoreporter', 'dessine-moi', 'téléthon', 'gamejam', 'faysal']  

# Create a regex pattern to match any of the words
pattern = '|'.join(words)

# Filter the DataFrame (case-insensitive)
df_ws_ml = df_ws_ml[df_ws_ml['Workshop'].str.contains(pattern, case=False, na=False)]

# To export this DataFrame to an Excel file, you can use the to_excel function:
df_ws_ml.to_excel('ML.xlsx')

In [43]:
# Group by 'TumoID' and 'PassStatus', then count the number of activities in each status
df_status_counts = df_ws_ml.groupby('TumoID')['PassStatus'].value_counts(dropna=False).unstack(fill_value=0)

# Renaming of count columns
df_status_counts = df_status_counts.rename(columns={'Fail': 'NumFailedMasterlabs', 'Pass': 'NumPassedMasterlabs', 'Withdrawn': 'NumWithdrawnMasterlabs', df_status_counts.columns[-1]: 'NumMasterlabsWithNullPassStatus'})

# Find number of enrolled workshops by student
df_status_counts['TotalNumEnrolledMasterlabs'] = df_status_counts['NumFailedMasterlabs'] + df_status_counts['NumPassedMasterlabs'] + df_status_counts['NumWithdrawnMasterlabs'] + df_status_counts['NumMasterlabsWithNullPassStatus']

df_status_counts.head()

# To export this DataFrame to an Excel file, you can use the to_excel function:
df_status_counts.to_excel('Masterlab Pass Status Aggregation.xlsx')

In [44]:
# Define a dictionary for attendance symbols
symbols_dict = {'AttendedMasterlab': '✅', 'NotAttendedMasterlab': '🔴', 'ExcusedMasterlab': '🚫', 'NotRecordedMasterlab': '🔲'}

# Define a function to count the occurrences of each symbol
def count_symbol(record, symbol):
    return record.count(symbol)

# Create additional columns for each symbol
for name, symbol in symbols_dict.items():
    df_ws_ml[name + 'SessionsCount'] = df_ws_ml['Attendance'].apply(lambda record: count_symbol(record, symbol))
    
def count_stars(x):
    return (x == True).sum()    

df_ws_ml.tail()

# Aggregate counts for session attendance and stars awarded
df_ws_ml_agg = df_ws_ml.groupby('TumoID').agg({
    'AttendedMasterlabSessionsCount': 'sum',  # Count
    'NotAttendedMasterlabSessionsCount': 'sum',  
    'ExcusedMasterlabSessionsCount': 'sum', 
    'NotRecordedMasterlabSessionsCount': 'sum',
    'Star': count_stars
})

# Renaming of count columns
df_ws_ml_agg = df_ws_ml_agg.rename(columns={'Star': 'MasterlabStars'})

df_ws_ml_agg.to_excel('Masterlab Session Attendance Aggregation.xlsx')

In [45]:
# Join dfs containing workshop/masterlab data
files = ['Masterlab Pass Status Aggregation.xlsx', 'Masterlab Session Attendance Aggregation.xlsx']

# Read the first file
df = pd.read_excel(files[0])

# Loop over the rest of the files and merge
for file in files[1:]:
    df_other = pd.read_excel(file)
    df = pd.merge(df, df_other, on='TumoID', how='outer')

# Save the merged DataFrame to a new Excel file
df.to_excel('ready/masterlab_data.xlsx', index=False)